# Library


In [1]:

import os
from config import get_config,update_config
import numpy as np
import torch

try:
    from apex import amp
except ImportError:
    print("AMP is not available")
from datasets.predict_loader import test_loader
from models import get_model, load_pretrained
from utils import get_memory_format



AMP is not available


In [3]:
def predict_videos(vid_dirs, save_dir, device="cuda",freq):
# vid_dirs: A list, contains all videos address
# ====== Output =======
for vid in vid_dirs:
    save_sub_dir="videoname without ext"
    predict_video(vid,save_sub_dir,device=device,freq=freq)


def predict_video(vid_dir, save_dir, device="cuda",freq):
# vid_dirs: video address (absolute)
# ====== Output =======
get_videotype(vid_dir)
print(detect video xxx, print meta information)

create a  file called temp under save_dir
extract video2Image(vid_dir, temp, freq)

predict_image(temp, "ALL", plot=True, heatmap=True, Demo=True, device="cuda")






IndentationError: expected an indented block after function definition on line 1 (517861323.py, line 4)

# Function

In [2]:
def predict_image(dataset_dir, image_list, device="cuda",plot=True, heatmap=True, Demo=True):

# dataset_dir: The Dir should in style of gazefollow, that contains head_information.txt and file named image_original contains all images
# checkpoint_dir: The dir of model weight.
# image_list: A list of image name, should be same with first column in file test_annotations_release.txt. If image_name=“ALL”, it process all image detected.
# ====== Output =======
# if plot, only plot the result 2 image per row;
# if heatmap, save heatmap prediction to heatmap_predict under dataset_dir;
# if demo, save square diagram into heatmap_predict under dataset_dir.


# Get config
    config= get_config()
    config= update_config(config,dataset_dir, device)
    device = torch.device(config.device)
    print(f"Running on {device}")

# Make Datasets
    print("Loading dataset")
    target_test_loader = test_loader(config,image_list)

# Load model
    print("Loading model")
    model = get_model(config, device=device)
    print(model)
    pretrained_dict = torch.load(config.eval_weights, map_location=device)
    pretrained_dict = pretrained_dict.get("model_state_dict") or pretrained_dict.get("model")
    model = load_pretrained(model, pretrained_dict)


In [ ]:
#predict_image(r"D:\Datasets\engagement_follow",["image_original/00000004.jpg"])

#predict_image(r"D:\Datasets\engagement_follow",["image_original/00000004.jpg"])

# Prediction

In [6]:
model.eval()
output_size = config.output_size
print_every = config.print_every

gaze_inside_all = []
gaze_inside_pred_all = []

with torch.no_grad():
    for batch, data in enumerate(target_test_loader):
        (
            images,
            depths,
            faces,
            head_channels,
            _,
            eye_coords,
            gaze_coords,
            gaze_inside,
            img_size,
            path,
        ) = data

        images = images.to(device, non_blocking=True, memory_format=get_memory_format(config))
        depths = depths.to(device, non_blocking=True, memory_format=get_memory_format(config))
        faces = faces.to(device, non_blocking=True, memory_format=get_memory_format(config))
        head = head_channels.to(device, non_blocking=True, memory_format=get_memory_format(config))
        gaze_inside = gaze_inside.to(device, non_blocking=True, memory_format=get_memory_format(config))

        gaze_heatmap_pred, gaze_inside_pred, _, _, _ = model(images, depths, head, faces)

        gaze_inside_all.extend(gaze_inside.cpu().tolist())
        gaze_inside_pred_all.extend(gaze_inside_pred.squeeze(1).cpu().tolist())
        gaze_heatmap_pred = gaze_heatmap_pred.squeeze(1).cpu()

        import os
        import cv2
        import torchvision.transforms as transforms
        for i, image_path in enumerate(path):
                # Extracting file name from the path
                file_name = os.path.basename(image_path)  # Gets '123.jpg' from 'test/123.jpg'

                # Define the directory where you want to save the predicted images
                save_dir = config.save_dir
                os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

                # Check if the file already exists. If so, modify the file name
                base_name, ext = os.path.splitext(file_name)  # Gets '123' and '.jpg' from '123.jpg'
                count = 0
                while os.path.exists(os.path.join(save_dir, file_name)):
                    count += 1
                    file_name = f"{base_name}_{count}{ext}"  # Modifies '123.jpg' to '123_1.jpg' etc.

                # Saving the gaze_heatmap_pred image
                heatmap_image = gaze_heatmap_pred[i]  # Assuming gaze_heatmap_pred has per-image predictions in the first dimension

                # Convert the heatmap tensor to an image and save it
                heatmap_image = transforms.ToPILImage()(heatmap_image)
                heatmap_image.save(os.path.join(save_dir, file_name))

                # Create an empty array to store the final composite image
                composite_img = np.zeros((448, 448, 3), dtype=np.uint8)


                mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
                std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)

                # left up
                original_image = images[i].cpu() * std + mean
                original_image = original_image[0][[0, 1, 2], :, :]
                original_image = transforms.ToPILImage()(original_image).convert("RGB")
                original_image = np.array(original_image.resize((224, 224)))
                heatmap_image_resized = heatmap_image.resize((224, 224))
                heatmap_image_resized = np.array(heatmap_image_resized)
                heatmap_image_resized = np.stack([heatmap_image_resized * 1, heatmap_image_resized* 1, heatmap_image_resized * 0], axis=2)
                masked_image = cv2.addWeighted(original_image, 1, heatmap_image_resized, 2, 0)
                composite_img[0:224, 0:224, :] = masked_image



                # LEFT DOWN: Depths image
                depths_image = transforms.ToPILImage()(depths[i].cpu()).convert("RGB")
                depths_image = np.array(depths_image.resize((224, 224)))
                composite_img[224:448, 0:224, :] = depths_image

                # RIGHT UP: Faces image
                faces_image = faces[i].cpu()* std + mean
                faces_image = faces_image[0][[0, 1, 2], :, :]
                faces_image = transforms.ToPILImage()(faces_image).convert("RGB")
                faces_image = np.array(faces_image.resize((224, 224)))
                composite_img[0:224, 224:448, :] = faces_image

                # RIGHT DOWN: Head_channels image (grayscale)
                head_channels_image = transforms.ToPILImage()(head_channels[i].cpu()).convert("L")
                head_channels_image = np.array(head_channels_image.resize((224, 224)))
                composite_img[224:448, 224:448, 0] = head_channels_image
                composite_img[224:448, 224:448, 1] = head_channels_image
                composite_img[224:448, 224:448, 2] = head_channels_image
                # Save composite_img
                composite_file_name = f"composite_{file_name}"
                cv2.imwrite(os.path.join(save_dir, composite_file_name), cv2.cvtColor(composite_img, cv2.COLOR_RGB2BGR))

In [7]:
print(heatmap_image)


<PIL.Image.Image image mode=L size=64x64 at 0x24454732810>
